A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_transformed that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (bool).
- Columns containing integers only must be stored as 32-bit integers (int32).
- Columns containing floats must be stored as 16-bit floats (float16).
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
- The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

If you call .info() or .memory_usage() methods on ds_jobs and ds_jobs_transformed after you've preprocessed it, you should notice a substantial decrease in memory usage.

In [100]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.enrolled_university.value_counts()

enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64

In [97]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# 1
ds_jobs_transformed['relevant_experience'] = np.where(ds_jobs_transformed['relevant_experience'] == 'Has relevant experience', True, False)

# 2, 3, 4
ds_jobs_transformed = ds_jobs_transformed.astype({'job_change': 'bool', 'relevant_experience': 'bool', 
                                                  'student_id': 'int32', 'training_hours':'int32', 
                                                  'city_development_index': 'float16',
                                                  'city' : 'category', 'gender': 'category', 'major_discipline': 'category', 'enrolled_university' : 'category', 'company_type': 'category', 
                                                  'education_level': 'category', 'experience': 'category', 'last_new_job': 'category', 'company_size': 'category'})

# 5
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.reorder_categories(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered = True)
experience_cat = ['<1'] + list(map(lambda x: str(x), list(range(1, 21)))) + ['>20']
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].cat.reorder_categories(experience_cat, ordered = True)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(['never', '1', '2', '3', '4', '>4'], ordered = True)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].cat.reorder_categories(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered = True)

# 6
ds_jobs_transformed = ds_jobs_transformed.loc[(ds_jobs_transformed['company_size'] > '500-999') & (ds_jobs_transformed['experience'] > '9')]

Index                       132
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64